### **Test the model**

In [1]:
%pip install -q donut-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("iarkh/donut-demo")
model = VisionEncoderDecoderModel.from_pretrained("iarkh/donut-demo")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/iarkh/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import re
import json
import torch
from tqdm.auto import tqdm
import numpy as np

from donut import JSONParseEvaluator

from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()
model.to(device)

output_list = []
accs = []

dataset = load_dataset("naver-clova-ix/cord-v2", split="validation")

for idx, sample in tqdm(enumerate(dataset), total=len(dataset)):
    # prepare encoder inputs
    pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    # prepare decoder inputs
    task_prompt = "<s_cord-v2>"
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
    decoder_input_ids = decoder_input_ids.to(device)

    # generate sequence
    outputs = model.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=model.decoder.config.max_position_embeddings,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            use_cache=True,
            num_beams=1,
            bad_words_ids=[[processor.tokenizer.unk_token_id]],
            return_dict_in_generate=True,
        )

    # turn into JSON
    seq = processor.batch_decode(outputs.sequences)[0]
    seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
    seq = processor.token2json(seq)

    ground_truth = json.loads(sample["ground_truth"])
    ground_truth = ground_truth["gt_parse"]
    evaluator = JSONParseEvaluator()
    score = evaluator.cal_acc(seq, ground_truth)

    accs.append(score)
    output_list.append(seq)

scores = {"accuracies": accs, "mean_accuracy": np.mean(accs)}
print(scores, f"length : {len(accs)}")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  0%|          | 0/100 [00:00<?, ?it/s]

/home/iarkh/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


{'accuracies': [1.0, 1.0, 0.5955056179775281, 0.7176470588235294, 0.9, 0.56, 1.0, 0.7236842105263157, 0.9285714285714286, 0.9166666666666666, 0.9696969696969697, 0.9808917197452229, 0.7857142857142857, 1.0, 0.912621359223301, 0.6695652173913044, 1.0, 0.9230769230769231, 0.9204545454545454, 1.0, 0.9852941176470589, 0.8379888268156425, 0.78, 0.9454545454545454, 1.0, 0.7903930131004366, 0.9753086419753086, 0.9838709677419355, 1.0, 0.5396825396825398, 0.975, 1.0, 1.0, 0.8275862068965517, 0.9675324675324676, 1.0, 0.6979166666666667, 0.9642857142857143, 0.9777777777777777, 0.9448275862068966, 0.34306569343065696, 0.7844311377245509, 1.0, 0.6582278481012658, 0.967741935483871, 1.0, 1.0, 0.92, 1.0, 0.9156626506024097, 1.0, 1.0, 0.9540229885057472, 0.8229166666666666, 0.968421052631579, 0.4, 0.5849056603773585, 0.9916666666666667, 0.9242424242424242, 0.9087301587301587, 0.8181818181818181, 0.7727272727272727, 0.7425742574257426, 0.898989898989899, 1.0, 0.967032967032967, 1.0, 0.7567567567567568

In [4]:
print("Mean accuracy:", np.mean(accs))
print("OK")

Mean accuracy: 0.8700536324536051
OK


In [ ]:
! pip install -q numpy==1.17.3 Cython google-colab

: 

### **Check my picture**

In [8]:
from google.colab import files
import numpy as np
from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
    file_path = fn

image = Image.open(file_path)
image_array = np.array(image)
image

ModuleNotFoundError: No module named 'google.colab'